# Problem Statement: Shakespearean Text Generation with Recurrent Neural Networks (RNNs)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import string
import random

In [ ]:
#kaggle download
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


In [ ]:
!kaggle datasets download -d kingburrito666/shakespeare-plays

 88% 4.00M/4.55M [00:00<00:00, 38.1MB/s]
100% 4.55M/4.55M [00:00<00:00, 42.1MB/s]


In [ ]:
from zipfile import ZipFile
file_name = "/content/shakespeare-plays.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
# Load and preprocess the Shakespeare dataset
with open('/content/alllines.txt', 'r', encoding='utf-8') as file:
    data = file.read()

# Create a mapping of unique characters to integers
chars = sorted(list(set(data)))
char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}

# Convert the text into integer sequences
int_data = [char_to_index[char] for char in data]

In [ ]:
# Define the RNN model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        # Check if hidden is not None before cloning
        if hidden is not None:
            hidden = hidden.clone()
        output, hidden = self.rnn(x, hidden)
        output = self.fc(output)
        return output, hidden


# Hyperparameters
input_size = len(chars)
hidden_size = 128
output_size = len(chars)
seq_length = 100  # Adjust as needed
learning_rate = 0.01


In [ ]:
# Initialize the model, loss function, and optimizer
rnn_model = SimpleRNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=learning_rate)

# Training the RNN
num_epochs = 1 # Adjust as needed
for epoch in range(num_epochs):
    hidden_state = None
    for i in range(0, len(int_data) - seq_length, seq_length):
        inputs = torch.tensor(int_data[i:i+seq_length])
        targets = torch.tensor(int_data[i+1:i+seq_length+1]) # Predict the next character
        optimizer.zero_grad()
        outputs, hidden_state = rnn_model(inputs, hidden_state)
        loss = criterion(outputs.view(-1, output_size), targets.view(-1))
        loss.backward()
        optimizer.step()
        # Detach the hidden state to prevent RuntimeError
        hidden_state = hidden_state.detach()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



In [ ]:
# Text generation using the trained RNN
def generate_text(model, seed, length=500):
    model.eval()
    generated_text = seed
    for _ in range(length):
        seed_encoded = torch.tensor([char_to_index[char] for char in seed])
        output, _ = model(seed_encoded, None)
        probabilities = torch.softmax(output[-1], dim=0).detach().numpy()
        predicted_index = random.choices(range(len(chars)), weights=probabilities)[0]
        seed += index_to_char[predicted_index]
        generated_text += index_to_char[predicted_index]
    return generated_text

In [ ]:
# Generate and print text
seed_text = "shall i compare thee to a summer's day?\n"
generated_text = generate_text(rnn_model, seed_text)
print("\nGenerated Text:")
print(generated_text)


Generated Text:
shall i compare thee to a summer's day?
"Andinto hing.mave by he I I wifscently I for hont I on I honred,"
"Mace, prale dar ny bo lo I CE I pren"
"Les honingstola,"
"And mure madeen han--he That hold I'll I Thowis how is eveyt, jace'd pand got. han"
"Yout hor fur upot, I I and bemadlinly prin tour witand I have didmelibaitaman p beeve fikead to madly I p Gacomboyed tohatand. wis ind ho I ach and sur by hold hon pain tanbid I I to to what, shout I pehone you's way,"
"And a hower's no whrace ant and hin's by nasturt I by vaw thom by ine
